# 03_batch_convert_folder.ipynb


## Aufgabe

In diesem Notebook sollen alle HTML-Dateien in einem Ordner automatisch verarbeitet werden.

Ziel: Automatisierte Konvertierung mehrerer HTML-Dateien mit einer einheitlichen Konfiguration, um Zeit zu sparen und konsistente Ergebnisse zu erzielen.


## Bedienung

1. Quellordner auswählen (Standard: `data/original/`)
2. Zielordner auswählen (Standard: `data/output/`)
3. Konvertierungsfunktion auswählen:
   - Bottom zu Top umrechnen
   - Offset anwenden
4. Parameter für die gewählte Funktion einstellen
5. Batch-Verarbeitung per Button starten
6. Ergebnisse werden im Zielordner gespeichert


## Technische Konstanten

- HTML_WIDTH = 1210
- HTML_HEIGHT = 825
- HTML_MARGIN_TOP = 0
- HTML_MARGIN_BOTTOM = 0


In [ ]:
# Benötigte Bibliotheken importieren
import sys
import os
import datetime
import glob

# Pfad zum shared-Verzeichnis hinzufügen, falls es nicht im Pythonpath ist
sys.path.append('..')

# Importieren der gemeinsam genutzten Funktionen
from shared.html_utils import (
    convert_bottom_to_top, 
    apply_offset, 
    batch_convert_folder,
    load_html_from_file, 
    save_html_to_file
)
from shared.constants import HTML_HEIGHT, HTML_WIDTH


In [ ]:
# Installieren von benötigten Paketen, falls nicht vorhanden
try:
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")
except ImportError:
    print("BeautifulSoup ist nicht installiert. Installation wird gestartet...")
    !pip install beautifulsoup4
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")
except ImportError:
    print("ipywidgets ist nicht installiert. Installation wird gestartet...")
    !pip install ipywidgets
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")


In [ ]:
# Verfügbare HTML-Dateien im Quellordner anzeigen
def list_html_files(folder_path="../data/original"):
    """Listet alle HTML-Dateien im angegebenen Ordner auf."""
    html_files = glob.glob(os.path.join(folder_path, "*.html"))
    return [os.path.basename(f) for f in html_files]

# Quellordner überprüfen
source_folder = "../data/original"
html_files = list_html_files(source_folder)
print(f"Verfügbare HTML-Dateien in {source_folder}:")
for file in html_files:
    print(f"- {file}")

if not html_files:
    print("Keine HTML-Dateien gefunden. Bitte fügen Sie zuerst Dateien in den Ordner ein.")


In [ ]:
# Erstellen der Widgets für die Benutzeroberfläche
source_folder_input = widgets.Text(
    value='../data/original',
    placeholder='Pfad zum Quellordner',
    description='Quellordner:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

target_folder_input = widgets.Text(
    value='../data/output',
    placeholder='Pfad zum Zielordner',
    description='Zielordner:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

conversion_function = widgets.Dropdown(
    options=[
        ('Bottom zu Top umrechnen', 'convert_bottom_to_top'),
        ('Offset anwenden', 'apply_offset')
    ],
    value='convert_bottom_to_top',
    description='Funktion:',
    disabled=False,
)

offset_x = widgets.IntSlider(
    value=0,
    min=-100,
    max=100,
    step=1,
    description='Offset X:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

offset_y = widgets.IntSlider(
    value=0,
    min=-100,
    max=100,
    step=1,
    description='Offset Y:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

convert_button = widgets.Button(
    description='Batch-Verarbeitung starten',
    disabled=False,
    button_style='success',
    tooltip='Klicken, um alle Dateien zu verarbeiten',
    icon='check'
)

output = widgets.Output()

# Anzeigen der Widgets
print("Batch-Verarbeitung konfigurieren:")
display(source_folder_input)
display(target_folder_input)
display(conversion_function)
print("\nParameter für die Konvertierungsfunktion:")
display(offset_x)
display(offset_y)
display(convert_button)
display(output)


In [ ]:
# Funktion für den Konvertierungsbutton
def on_convert_button_clicked(b):
    with output:
        output.clear_output()
        
        # Ordnerpfade auslesen
        source_path = source_folder_input.value
        target_path = target_folder_input.value
        
        # Überprüfen, ob die Ordner existieren
        if not os.path.isdir(source_path):
            print(f"Fehler: Quellordner {source_path} existiert nicht.")
            return
        
        # Zielordner erstellen, falls er nicht existiert
        os.makedirs(target_path, exist_ok=True)
        
        # HTML-Dateien im Quellordner auflisten
        html_files = list_html_files(source_path)
        if not html_files:
            print(f"Keine HTML-Dateien in {source_path} gefunden.")
            return
        
        print(f"Starte Batch-Verarbeitung von {len(html_files)} Dateien...")
        print(f"Quellordner: {source_path}")
        print(f"Zielordner: {target_path}")
        
        # Konvertierungsfunktion und Parameter auswählen
        func_name = conversion_function.value
        offset_x_value = offset_x.value
        offset_y_value = offset_y.value
        
        # Konvertierungsfunktion auswählen
        if func_name == 'convert_bottom_to_top':
            print(f"Funktion: Bottom zu Top umrechnen mit Offset X: {offset_x_value}, Y: {offset_y_value}")
            func = lambda html: convert_bottom_to_top(html, offset_x_value, offset_y_value)
        elif func_name == 'apply_offset':
            print(f"Funktion: Offset anwenden mit X: {offset_x_value}, Y: {offset_y_value}")
            func = lambda html: apply_offset(html, offset_x_value, offset_y_value)
        
        # Batch-Verarbeitung durchführen
        processed_files = []
        for html_file in html_files:
            input_path = os.path.join(source_path, html_file)
            
            # HTML-Datei laden
            html_string = load_html_from_file(input_path)
            if not html_string:
                print(f"Fehler beim Laden von {html_file}, überspringe...")
                continue
            
            # Konvertierung durchführen
            converted_html = func(html_string)
            
            # Ausgabedateiname generieren
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
            output_filename = f"{func_name}_{os.path.splitext(html_file)[0]}_{timestamp}.html"
            output_path = os.path.join(target_path, output_filename)
            
            # Konvertierte Datei speichern
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(converted_html)
            
            processed_files.append(output_path)
            print(f"Verarbeitet: {html_file} -> {output_filename}")
        
        print(f"\nBatch-Verarbeitung abgeschlossen. {len(processed_files)} Dateien verarbeitet.")
        print(f"Alle konvertierten Dateien wurden in {target_path} gespeichert.")

# Button-Klick-Event registrieren
convert_button.on_click(on_convert_button_clicked)


## Alternative: Verwendung der batch_convert_folder-Funktion

Die obige Implementierung zeigt den detaillierten Prozess der Batch-Verarbeitung. Alternativ kann auch die vordefinierte `batch_convert_folder`-Funktion aus dem shared-Modul verwendet werden:


In [ ]:
# Beispiel für die Verwendung der batch_convert_folder-Funktion
def use_batch_function():
    # Parameter definieren
    input_folder = "../data/original"
    output_folder = "../data/output"
    offset_x_value = 5
    offset_y_value = -10
    
    # Batch-Konvertierung mit convert_bottom_to_top
    print("Batch-Konvertierung mit convert_bottom_to_top:")
    converted_files = batch_convert_folder(
        input_folder=input_folder,
        output_folder=output_folder,
        conversion_function=convert_bottom_to_top,
        offset_x=offset_x_value,
        offset_y=offset_y_value
    )
    
    print(f"{len(converted_files)} Dateien verarbeitet.")
    
    # Batch-Konvertierung mit apply_offset
    print("\nBatch-Konvertierung mit apply_offset:")
    converted_files = batch_convert_folder(
        input_folder=input_folder,
        output_folder=output_folder,
        conversion_function=apply_offset,
        offset_x=offset_x_value,
        offset_y=offset_y_value
    )
    
    print(f"{len(converted_files)} Dateien verarbeitet.")

# Diese Funktion kann manuell ausgeführt werden, wenn die direkte Verwendung
# der batch_convert_folder-Funktion gewünscht ist
# use_batch_function()


## Erklärung der Batch-Verarbeitung

Die Batch-Verarbeitung ermöglicht es, mehrere HTML-Dateien mit einer einheitlichen Konfiguration zu konvertieren. Dies ist besonders nützlich, wenn:

1. Viele Dateien mit den gleichen Einstellungen konvertiert werden müssen
2. Regelmäßige Konvertierungen mit festen Parametern durchgeführt werden
3. Eine konsistente Verarbeitung aller Dateien sichergestellt werden soll

Der Prozess umfasst folgende Schritte:
1. Laden aller HTML-Dateien aus dem Quellordner
2. Anwenden der ausgewählten Konvertierungsfunktion mit den definierten Parametern
3. Speichern der konvertierten Dateien im Zielordner mit eindeutigen Namen

Die Batch-Verarbeitung spart Zeit und reduziert die Fehleranfälligkeit bei der manuellen Konvertierung vieler Dateien.